# Create covariate stacks

### Covariates will be derived by reducing satellite imagery to useful modelling layers
#### Covariates can include: Landsat 8, NASADEM, Sentinel-2, Sentinel-1

### Inputs:
#### bbox_img_tile: An image tile extent will provide the bbox that accesses cloud-optimized GeoTiffs
#### filters: search for imagery based on days-of-year, cloudcover, year range
##### Note: Image tile extent will be dictated by data volume of covars & CPU RAM

In [1]:
# Search for imagery
# https://github.com/developmentseed/example-jupyter-notebooks/blob/landsat-search/notebooks/Landsat8-Search/L8-USGS-satapi.ipynb
import os
import json
import requests
import datetime

sat_api_url = "https://landsatlook.usgs.gov/sat-api"

def query_satapi(query):
    headers = {
            "Content-Type": "application/json",
            "Accept-Encoding": "gzip",
            "Accept": "application/geo+json",
        }

    url = f"{sat_api_url}/stac/search"
    data = requests.post(url, headers=headers, json=query).json()
    
    return data

def query_year(year, bbox, min_cloud, max_cloud):
    '''Given the year, finds the number of scenes matching the query and returns it.'''
    date_min = '-'.join([str(year), "06-01"])
    date_max = '-'.join([str(year), "09-15"])
    start_date = datetime.datetime.strptime(date_min, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(date_max, "%Y-%m-%d") 
    start = start_date.strftime("%Y-%m-%dT00:00:00Z")
    end = end_date.strftime("%Y-%m-%dT23:59:59Z")
    
    query = {
    "time": f"{start}/{end}",
    "query": {
        "bbox":bbox,
        "eo:cloud_cover": {"gte": min_cloud, "lt": max_cloud},
        "collection":{"eq": "landsat-c2l2-sr"}
        },
    "limit": 500 # We limit to 500 items per Page (requests) to make sure sat-api doesn't fail to return big features collection
    }
    
    data = query_satapi(query)
    scenes = data['meta']['found']
    
    return scenes

In [11]:
# Accessing imagery
# Select an area of interest
bbox_list = [[-105,45,-100,50], [-101,45,-100,46]]
min_cloud = 0
max_cloud = 20
for bbox in bbox_list:
    # Geojson of total scenes - Change to list of scenes
    scene_totals = [query_year(year, bbox, min_cloud, max_cloud ) for year in range(2015,2020 + 1)]
    print(scene_totals)

[46585, 45151, 47858, 47872, 48947, 50710]
[46585, 45151, 47858, 47872, 48947, 50710]


In [2]:
bbox_list = [[-105,45,-100,50], [-101,45,-100,46]]
min_cloud = 0
max_cloud = 20

In [5]:
scene_totals = query_year(2020, bbox_list[1], min_cloud, max_cloud) 
scene_totals

50710

In [ ]:
# Processing imagery (band math)
# is this done on the fly? No

# Authenticate to aws to access a requester pays bucket
aws_session = AWSSession(boto3.Session(aws_access_key_id=aws_access_key_id,
                                      aws_secret_access_key=aws_secret_access_key),                           requester_pays=True)

In [ ]:
# Adding processed imagery to a covariate stack